In [2]:
import librosa
import numpy as np
import os
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
import pickle
%run feature_extraction.ipynb

In [3]:
'''
>>> Açıklama
>>> --------
>>> Bu fonksiyon ile ses verileri KNN algoritması ile eğitilir.
>>> Eğitim verileri training_set klasöründen alınır.
>>> Eğitilen veriler training_models klasörüne kaydedilir.
>>> Eğitilen verilerin doğruluk oranı ekrana yazdırılır.
'''
def train_model():
    if not os.path.exists("training_models"):
            os.makedirs("training_models")
            
    # Eğer training_set klasörü yoksa hata ver ve programı sonlandır
    if not os.path.exists("training_set"):
        print("Eğitilecek veri bulunamadı!")
        exit()
            
    features = []
    labels = []

    dest = "training_models"
    trainFolder = "training_set"
    trainActorFolder = os.listdir(trainFolder)

    for actor in trainActorFolder:
        # klasördeki tüm wav dosyalarını al
        files = [os.path.join(trainFolder, actor, file) for file in os.listdir(os.path.join(trainFolder, actor)) if file.endswith(".wav")]
        
        for file in files:
            fullname = file.strip().split('\\')[1].split('.')[0]
            audio, rate = librosa.load(file)

            # ses dosyasının başındaki ve sonundaki boşlukları kaldır
            y_trimmed, _ = librosa.effects.trim(audio, top_db=20)
            
            # ses dosyasını 200 milisaniyelik parçalara böl
            # her bir parçadan mfcc çıkar
            for i in range(0, len(y_trimmed), 4410):
                if i+4410 < len(y_trimmed):
                    #extract mfcc
                    comph_mfccs = feature_extraction(y_trimmed[i:i+4410], rate)
                    features.append(comph_mfccs)
                    labels.append(fullname)

    features = np.array(features).reshape(len(features), -1)
    labels = np.array(labels)

    # verileri eğitim ve test verisi olarak ayır
    train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size=0.2, random_state=42)

    # KNN algoritması ile eğit
    knn = KNeighborsClassifier(n_neighbors=3)
    knn.fit(train_features, train_labels)

    test_features = test_features.reshape(len(test_features), -1)

    # doğruluk oranını hesapla
    print("Eğitim verisi doğruluk oranı: ", knn.score(train_features, train_labels))
    print("Test verisi doğruluk oranı: ", knn.score(test_features, test_labels))

    # sınıflandırma raporu ve karışıklık matrisi
    print("\nSınıflandırma raporu: \n", metrics.classification_report(test_labels, knn.predict(test_features)))

    # fm ve precision değerlerini hesapla
    print("\nF1 score: ", f1_score(test_labels, knn.predict(test_features), average='macro'))
    print("Precision score: ", precision_score(test_labels, knn.predict(test_features), average='macro'))


    # modeli kaydet
    import pickle
    filename = os.path.join(dest, 'finalized_model.sav')
    pickle.dump(knn, open(filename, 'wb'))

    print("\nModel başarıyla kaydedildi!")

In [5]:
train_model()

Eğitim verisi doğruluk oranı:  0.9758358159549818
Test verisi doğruluk oranı:  0.9563492063492064

Sınıflandırma raporu: 
               precision    recall  f1-score   support

       ahmet       0.52      0.39      0.45        28
     testing       0.78      0.88      0.82        16
       yusuf       0.97      0.98      0.98       712

    accuracy                           0.96       756
   macro avg       0.76      0.75      0.75       756
weighted avg       0.95      0.96      0.95       756


F1 score:  0.749805310507688
Precision score:  0.7583626663124571

Model başarıyla kaydedildi!
